## Common pattern continued

### State Reduction
* The number of states is the product of the number of values of each state variable
  + reducing the states usually reduces the time and space complexity
* Several strategies to reduce the number of states
  + find the relationship between state variables and reduce the number of state variables 
  + reduce the number of state variables from recurrent equation (House robber problem, we remove the rob/nonrob state variable and only used the current house index as the state variable)
  + improve space complexity when the recurrence relation is static (Fibonacci, the current value only depends on the previous two, so we only need to store the previous two values to reduce space complexity)
  
  ```ptyhon
  class Solution:
    def fib(self, n: int) -> int:
        if n <= 1: return n
        one_back = 1
        two_back = 0
        for i in range(2, n + 1):
            temp = one_back
            one_back += two_back
            two_back = temp

        return one_back
   ```

### Leetcode 746 Min Cost Climbing Stairs
* refer to this problem in Strategic Approach to DP notebook

### Counting DP
* A class of DP problems to ask for the number of distinct ways to do something
* difference between counting DP and other DP problems asking for min and max
  + recurrence relationship
    + in DP problems asking for min and max, we usually use the recurrence relationship of max() or min()
    + in counting DP problems, the recurrence relationship typically just sums the results of multiple states together
  + base case
    + in other DP problems, we usually define base cases to return 0
    + in counting DP, we use logic to find the reasonable base case value, for example, in Leetcode 70, Climbing Stairs, we set the base case when i=0 as 1 since there is one way to climb the stair 0

### Leetcode 70 Climbing Stairs
* overview
  + you are climbing a staircase. It takes n steps to reach the top
  + each time, you can climb 1 or 2 steps, how many distince ways can you climb to the top?  

In [1]:
# bottom up with state reduction
class Solution:
    def climbStairs(self, n: int) -> int:
        prev_two, prev_one = 1, 2

        if n <=2 :
            return n

        for _ in range(3, n+1):
            prev_one, prev_two = prev_one + prev_two, prev_one
            
        return prev_one

### Leetcode 276 Paint Fence
* overview
  + You are painting a fence of n posts with k different colors. you much paint the posts following these rules:
    + Every post must be painted exactly one color
    +  There cannot be three or more consecutive posts with the same color
  + given the two integer n and k, return the number of ways you can paint the fence
* algorithm
  + at the begining, when n = 1, we can use all the k colors, resulting in k ways
  + when n = 2, we can still use k colors since we are restricted to have three same colors, so we will get k^2 ways
    + within these k^2 ways, there are k ways corresponding to the same colors, and k(k-1) ways corresponding to different colors
    + we store the number of ways using the same colors as the previous post in a variable called 'same'
    + we store the number of ways using the different colors than the previous post in variable 'diff'
  + starting from n = 3, we can have two strategies
    + choose a color different from the previous post, which we can have k-1 colors to choose, these k-1 choices can combine with any of the color combination of the previous post, and thus, diff = (k-1)(previous diff + previous same)
    + choose a color same as the previous post. For this option, we have diff combinations
    + the total number of combination for this n is diff+same  

In [1]:
class Solution:
    def numWays(self, n: int, k: int) -> int:
        if n == 1:
            return k
        if n == 2:
            return k*k
        
        # the 2nd post can be separated into two cases:
        # select the same color as the previous one, same=k, and 
        # select the different color than the previous one, diff = (k-1)*k
        diff = k*(k-1)
        same = k
                
        for _ in range(3, n+1):
            same, diff = diff, (k-1)*(diff+same)
            
        return same + diff           

### Leetcode 518 Coin Change II
* overview
  + you are given an integer array coins representing coins of different denominations and an interger amount representing a total amount of money
  + return the number of combinations that make up that amount. If that amount of money cannot be made up by any combination of the coins, return 0
  + You may assume that you have an infinite number of each kind of coin.

In [1]:
from typing import List
class Solution:
    def change(self, amount: int, coins: List[int]) -> int:
        if not coins:
            return 0
        
        dp = [0] *(amount+1)
        
        # when amount == 0, there is one way by using no coins
        dp[0] = 1
        
        # we traverse coins rather than dp index (amount) to prevent 
        # permutations, rather than combinations to be counted.
        # if we use i loop as the outer loop, then for i ==10, (3, 7)
        # and (7, 3) will be counted twice
        for coin in coins:
            for i in range(coin, amount+1):
                dp[i] += dp[i-coin]
                    
        return dp[-1]             

### Leetcode 91 Decode ways
* overview
  + A message containing letters from A-Z can be encoded into numbers using the mapping of A-> 1 and Z-> 26.
  + given a string of numbers, find how many different ways we can decode it back to letters
* algorithm:
  + we have a static recurrent relationship, that is the current result at index i depends on the sum of results at i+1 and i+2
  + we first test the edge case where n ==0 and n==1
  + set one_after = 1 and two_after = 0
  + here one_after is the base case corresponding to the letter after the last letter in the string. This is an imaginary letter for other index to accumulate from 1. two_after refers to the ways corresponding to the letter two chars after the current letter
  + we then iterate from n-1 to 0 (includes 0) to traverse the current letter and return the curr corresponding to i=0
  + for each iteration
    + first initialize the curr = 0
    + add curr by one_after 
    + we then test if the current letter concat the letter after it can be converted to a number betwee \[10, 26\], if so
      + increment curr by two_after
    + the algorithm's logic is that how many ways to decode the letter from the last letter to the current (including) is the sum of the ways to the letter exactly after it, and the letter that are two letters after it, if applicable
      + note that the current letter doesn't create any new combinations, it just inherit the ways from the letters after it. This is basically the same as coin changes problem (Leetcode 518)
* time complexity O(n) by traversing the n letters
* space complexity O(1) since we only need curr, one_after and two_after variables

In [2]:
class Solution:
    def numDecodings(self, s: str) -> int:
        if not s:
            return 0
        n = len(s)
        
        if n == 1:
            return 0 if s =="0" else 1
        
        # one_after refers to substring starting with empty string
        # at index n, and after. 
        one_after, two_after = 1, 0
        curr = 0
        
        
        # i refers to the starting index to the end
        # and how many ways of this substring can 
        # be decoded. If current letter is 0, then
        # any substring starts with 0 has 0 ways to decode
        # any letter j before a zero will have to check j+2 
        for i in range(n-1, -1, -1):
            curr = 0
            
            if s[i] != "0":
                curr += one_after
                if i < n-1 and 9 < int(s[i] + s[i+1]) < 27:
                    curr += two_after
                    
            one_after, two_after = curr, one_after
            
        return curr     
        

### Kadan's Algorithm
* an algorithm to find the max sum subarray given an array of numbers in O(n) time and O(1) space
* the algorithm iterates through the array using an integer variable current, and at each index i, determines if elements before index i ar worth keeping, or should be discarded
* only useful when the array contains negative numbers. If current becomes negative, it is reset, and we start to consider a new subarray staring at the current index
* alogrithm
  + the basic logic is that if the subarray sum before the current element is negative, it won't help to build a subarray with higher sum, and we will start a new subarray from the current element, which will have a higher sum value
  + for each iteration, we use the best to record the historic highest sum
  + finally return best

### Leetcode 121 Best Time to Buy and Sell Stock
* overview
  + you are given an array prices where price(i) is the price of a give stock on the ith day
  + you want to maximize your profit by choosing a single day to buy one stock and choosing a different day in the future to sell that stock
  + return the maximum profit you can achieve from this transaction. If you can not achieve any profit, return 0
* algorithm
  + set rs = 0, this will be the historical highest profit
  + set curr_min = float("inf"), this represents the minimum price so far
  + the problem is to find the max positive difference between the current element and the minimum elment so far in the array
  + we apply Kadan's algorithm by scanning the elements, and store the max difference between the current element and the minimun element so far in the rs variable, then update the curr_min
  + we finally return the rs  

In [5]:
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        if not prices:
            return 0
        
        rs = 0
        curr_min = float("inf")
        
        for price in prices:
            rs = max(rs, price - curr_min)
            curr_min = min(curr_min, price)
            
        return rs    

### Leetcode 918 Maximum Sum Circular Subarray
* overview
  + given a circular integer array nums of length n, return the maximum possible sum of a non-empty subarray of nums
  + A circular array means the end of the array connects to the beginning of the array. Formally, the next element of nums\[i\] is nums\[(i + 1) % n\] and the previous element of nums\[i\] is nums\[(i - 1 + n) % n\].
  + A subarray may only include each element of the fixed buffer nums at most once.
* algorithm
  + this is an extension to the subarray max problem, with circular subarrays. There are two cases:
    + the subarray with max sum is within the array as a normal array
      + we can use the classic algorithm by cum_sum_max
    + the subarray consists of some elements at the end combined with some elements at the begining of the array (rewind sub array)
      + we the cum_sum_min, and the max sum will be the sum of the entire array minus the cum_sum_min
    + if the entire array consists of negative or zero elements, the total-cum_sum_min == 0, then we will not get the correct result. To exclude this edge case, we first check if cum_sum_max <=0, if so, we direct return it as the result.This is because if cum_sum_max <=0, then total - cum_sum_min will not make sense (the sum will be greater than total)
  

In [6]:
class Solution:
    def maxSubarraySumCircular(self, nums: List[int]) -> int:
        
        if not nums:
            return 0
        
        cum_sum_max = float("-inf")
        cum_sum_min = float("inf")
        
        # we don't have any element to sum, so curr_sum_max=curr_sum_min=0
        curr_sum_max = curr_sum_min = 0
        total = 0
        
        for num in nums:
            curr_sum_max = max(curr_sum_max+num, num)
            cum_sum_max = max(cum_sum_max, curr_sum_max)
            
            curr_sum_min = min(curr_sum_min+num, num)
            cum_sum_min = min(cum_sum_min, curr_sum_min)
            
            total += num
            
        if cum_sum_max <= 0:
            return cum_sum_max
        return max(cum_sum_max, total-cum_sum_min)
            